# 토큰 분류 과제 : 한국어 NER 예측 모델

교재 4장을 참고하여 한국어 NER 데이터셋에 대한 예측 모델을 만들어 보자

dataset: `klue`

모델: `skt/kogpt2-base-v2` (또는 다른 모델)

아래 Q에 해당하는 내용을 수행하자.

제출 내용
- ipynb 파일을 다운로드 하여 LMS에 제출
    - 본인의 수행 결과가 ipynb에 포함되게 저장하여 제출

마감일
- 5월 07일 자정까지

In [ ]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=4)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 588 (delta 5), reused 3 (delta 1), pack-reused 569
Receiving objects: 100% (588/588), 57.41 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (293/293), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.28.1
Using datasets v2.12.0
Using accelerate v0.18.0
Using sentencepiece v0.1.99
Using seqeval


깃허브 저장소 "NLP with Transformers" 를 클론하고 해당 디렉토리로 이동하는 것을 시도한다. 

In [ ]:
import pandas as pd
from itertools import accumulate
import numpy as np
from transformers import GPT2ForTokenClassification

In [78]:
from datasets import get_dataset_config_names

klue_subsets = get_dataset_config_names("klue")
print(f"KLUE 서브셋 개수: {len(klue_subsets)}")

KLUE 서브셋 개수: 8


Huggin Face의 'datasets'라이브러리를 사용하여 KLUE 데이터셋의 하위셋 이름을 가져와서 그 개수를 출력하는 것이다.

get_dataset_config_names() 함수를 호출하여 "klue"라는 문자열을 전달합니다.이 함수는 "klue"와 일치하는 모든 데이터셋 구성 이름을 반환합니다.

len() 함수를 사용하여 KLUE 데이터셋의 하위셋 개수를 구하고, 출력문을 사용하여 그 개수를 출력합니다. 이를 통해 KLUE 데이터셋의 하위셋이 몇 개인지 알 수 있습니다.

In [ ]:
from datasets import load_dataset

klue_datasets = load_dataset("klue", name="ner")

Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/ner/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Hugging Face의 datasets 라이브러리를 사용하여 KLUE 데이터셋의 Named Entity Recognition (NER) 하위셋을 로드하는 것입니다.

load_dataset() 함수를 호출하여 "klue" 데이터셋을 로드하고, name="ner" 인자를 사용하여 NER 하위셋을 선택합니다. 이 함수는 선택한 데이터셋과 해당 하위셋을 로드하고, 각각의 데이터를 딕셔너리 형태로 반환합니다.

In [ ]:
tags = klue_datasets['train'].features['ner_tags'].feature

index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

KLUE NER 데이터셋에서 태그 정보를 추출하여, 태그 이름을 인덱스에 매핑하고, 반대로 인덱스를 태그 이름에 매핑하는 두 개의 딕셔너리를 만드는 것

In [ ]:
from transformers import AutoTokenizer

In [ ]:
kogpt2_name = "skt/kogpt2-base-v2"

변수 kogpt2_name 에 'skt/kogpt2-base-v2' 라는 문자열을 할당하는 것이다.

skt에서 제공하는 KoGPT2 모델의 이름이다다

In [ ]:
gpt_tokenizer = AutoTokenizer.from_pretrained(kogpt2_name, add_prefix_space=True, 
                                              bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>'
                                             )

AutoTokenizer 클래스를 사용하여 kogpt2_name 으로 지정된 KoGPT2 모델의 기본 토크나이저를 불러와 gpt_tokenizer 변수에 할당하는 것이다.

from_pretrained() 메소드를 사용하여 사전 학습된 모델을 로드하며, add_prefix_space=True 인자를 사용하여 단어와 구분자 사이에 공백을 추가하여 토크나이저를 구성합니다.

In [ ]:
tags

ClassLabel(names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], id=None)

tags 변수는 KLUE NER 데이터셋에서 사용되는 개체명 태그 목록을 담고 있는 개체이다.

# 토크나이징 및 데이터 셋 가공

klue 의 ner 데이터
- 26k rows (train 21k, validaion 5k)

예제
- sentence
    ```
    특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다
    ```
- tokens:
    ```
    [ "특", "히", " ", "영", "동", "고", "속", "도", "로", " ", "강", "릉", " ", "방", "향", " ", "문", "막", "휴", "게", "소", "에", "서", " ", "만", "종", "분", "기", "점", "까", "지", " ", "5", "㎞", " ", "구", "간", "에", "는", " ", "승", "용", "차", " ", "전", "용", " ", "임", "시", " ", "갓", "길", "차", "로", "제", "를", " ", "운", "영", "하", "기", "로", " ", "했", "다", "." ]
    ```
- ner_tags:
    ```
    [ 12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12 ]
    ```

레이블
- 총 13개
    ```
    'B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'
    ```

교재 4장과 다른점
- xtreme 의 PAN-X.de 등은 공백 단위로 태깅 되어 있음 (즉, 공백 기준으로 단어가 나뉘고, 단어에 태깅되어 있음)
- klue의 ner 데이터 셋은 단어 내부에서 태깅이 종료될 수 있음 (예: **만종분기점**까지)
- 토크나이저가 인코딩한 토큰 문제
    - (예: 문막휴게소에서 -> _문, 막, 휴, 게, 소에서)
        - 토크나이징 수준에서 위와 같이 분리 되면, 문막휴게소만을 LC로 예측하는게 원천적으로 불가능하다.
        - 이경우 '소에서' 까지를 개체명으로 취급하여도 옳은 것으로 하자.
    - (예: 만종분기점까지 -> 만, 종, 분, 기점, 까지)
        - 이경우는 '까지를' 을 'O' 레이블로 명확히 분리하여, '만종분기점'까지를 LC로 예측 가능하다.

데이터 가공
- tokens를 모두 붙인 후 공백으로 분리하여 단어의 시퀀스로 문장을 취급하자.
- 이후 교재 4장과 동일하게 진행해보자.

참고
- konenizer 호출시 `is_split_into_words=True` 사용 

In [ ]:
def tokenize_and_align_labels(examples):
    wordified_sentences = [''.join(l_).split() for l_ in examples['tokens']]
    tokenized_inputs = gpt_tokenizer(wordified_sentences, truncation=True, is_split_into_words=True)

    # TODO: 교재 4장 참고하여 작성
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_labels = []
        for j, word in enumerate(wordified_sentences[i]):
            sub_tokens = gpt_tokenizer.tokenize(word)
            for st_idx in range(len(sub_tokens)):
                if st_idx == 0:
                    word_labels.append(label[j])
                else:
                    word_labels.append(12) # 'O' label
        labels.append(word_labels)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs                            

In [ ]:
ds = klue_datasets.map(tokenize_and_align_labels, batched=True, remove_columns=['sentence', 'tokens', 'ner_tags'])

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# 성능 측정

# 미세 튜닝하기

Q: 교재 4장을 참고하여 아래 실시
- TrainingArguments 설정
- Huggingface 로그인
- DataCollator 설정
- Trainder 설정
- 학습 실시
- 학습 종료후 허깅페이스 모델 허브에 푸시 실시
    - 결과 모델의 모델 허브에 업로드된 이름을 명시해야 함

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

현재 사용 가능한 CUDA 장치가 있을 경우 cuda를 그렇지 않은 경우 'cpu'를 선택하여 device 변수에 할당하는 코드이다.

GPU 를 사용할 수 있는 환경에서 모델 학습 시 더 빠른 속도를 얻을 수 있다.

In [ ]:
from transformers import TrainingArguments

num_epochs = 3
# 코랩에서 GPU 메모리 부족 에러가 나는 경우 batch_size를 16으로 줄여 주세요.
batch_size = 24  # 16
logging_steps = len(ds["train"]) // batch_size

# 주의: 상황에 맞게, model_name 수정할 것
model_name = f"{kogpt2_name}-finetuned-klue-ner"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch", 
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False, 
    logging_steps=logging_steps, push_to_hub=True)





*  num_epochs : 학습할 epoch 수
*  batch_size : 학습할 때 사용할 배치 크기
* logging steps: 학습 중에 loss 등을 logging 할 스텝 간격
* model name : 학습된 모델을 저장할 경로와 모델 이름
* TrainingArguments: 학습에 필요한 argument들을 저장한다. 
output_dir은 학습된 모델을 저장할 디렉토리. 
num_train_epochs는 학습할 epoch 수, per_device_train_batch_size는 학습할 때 사용할 배치 크기, per_device_eval_batch_size는 validation 할 때 사용할 배치 크기, evaluation_strategy는 validation을 언제 실행할지를 결정. save_steps는 몇 step마다 checkpoint를 저장할지 결정. weight_decay는 weight decay 계수. logging_steps는 학습 중에 logging할 step 간격. disable_tqdm은 progress bar를 보여줄지를 결정. push_to_hub는 학습된 모델을 Hugging Face Hub에 push할지를 결정.




In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # 레이블 IDs = -100 무시
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

모델이 예측한 개체명 태깅 결과와 실제 개체명 태깅 레이블을 비교하여 정확도를 측정하기 위해 사용

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

'seqeval'패키지에서 제공하는 'f1_score'함수를 이용해 평가 지표를 계산한다

'align_predictios'함수를 이용해 모델 예측값과 정답 레이블을 분리한 후 'f1_score'함수로 f1점수를 계산한다.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(gpt_tokenizer)

In [ ]:
from transformers import AutoConfig

kogpt2_config = AutoConfig.from_pretrained(kogpt2_name, 
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [ ]:
def model_init():
    return (GPT2ForTokenClassification
            .from_pretrained(kogpt2_name, config=kogpt2_config)
            .to(device))

적절한 device(cpu,gpu)으로 학습에데이터를 전송한다

In [ ]:
from transformers import Trainer

trainer = Trainer(model_init=model_init, args=training_args, 
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=ds["train"],
                  eval_dataset=ds["validation"], 
                  tokenizer=gpt_tokenizer)

Cloning https://huggingface.co/jooyy/kogpt2-base-v2-finetuned-klue-ner into local empty directory.


In [ ]:
trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.329400,0.271903,0.539024
2,0.204000,0.237410,0.614252
3,0.148300,0.199858,0.683178


Upload file pytorch_model.bin:   0%|          | 1.00/490M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.56k [00:00<?, ?B/s]

Upload file runs/May07_10-55-49_38a828cf4652/1683458531.8568668/events.out.tfevents.1683458531.38a828cf4652.96…

Upload file runs/May07_10-55-49_38a828cf4652/events.out.tfevents.1683458531.38a828cf4652.967.0:   0%|         …

To https://huggingface.co/jooyy/kogpt2-base-v2-finetuned-klue-ner
   2a33daf..93ede58  main -> main

   2a33daf..93ede58  main -> main

To https://huggingface.co/jooyy/kogpt2-base-v2-finetuned-klue-ner
   93ede58..0673668  main -> main

   93ede58..0673668  main -> main



'https://huggingface.co/jooyy/kogpt2-base-v2-finetuned-klue-ner/commit/93ede58765d06875cc547fe91be3258b01380844'

# 검증셋에 대해 확인

Q: validation set의 10개의 샘플에 대해
- 예측 레이블 얻고
- 이를 실제 레이블과 비교

In [ ]:
from datasets import Dataset
ts = Dataset.from_dict(ds['validation'][:10])

In [ ]:
predictions = trainer.predict(ts)

In [ ]:
r = align_predictions(predictions.predictions, predictions.label_ids)

# 추가 시도

Q: 검증 셋에 대한 성능을 더 끌어올리는 시도를 해보자
- 다른 모델을 조사 및 활용
- 에포크 등 하이퍼 파라미터 변형

* KoELECTRA


> 한국어 위키피디아와 나무위키를 학습한 전이학습 언어 모델. 구글의 ELECTRA 모델 구조를 사용하고 있으며, 네이버의 NER 과 같은 자연어처리 태스크에서 우수한 성능을 보여준다

* HanBERT


> SKT 에서 만든 한국어 BERT 모델이다. 네이버 쇼핑 리뷰 데이터셋으로 학습되었으며, 한국어 자연어 처리 태스크에서 좋은 성능을 보여준다

* KorBERT 

> SKT에서 만든 한국어 BERT 모델이다. 위키피디아 데이터셋을 사용하여 학습되었다.





In [70]:
from transformers import ElectraTokenizer, ElectraForTokenClassification, Trainer, TrainingArguments, pipeline
import torch

In [65]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForTokenClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [71]:
ner = pipeline("ner",model = model, tokenizer=tokenizer)

In [75]:
example = "오늘 날씨 알려줘"
ner_results = ner(example)
print(ner_results)

[{'entity': 'LABEL_0', 'score': 0.5697398, 'index': 1, 'word': '오늘', 'start':
None, 'end': None}, {'entity': 'LABEL_0', 'score': 0.5727621, 'index': 2,
'word': '날씨', 'start': None, 'end': None}, {'entity': 'LABEL_1', 'score':
0.50532633, 'index': 3, 'word': '알려', 'start': None, 'end': None}, {'entity':
'LABEL_0', 'score': 0.65141755, 'index': 4, 'word': '##줘', 'start': None, 'end':
None}]
